In [1]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

from federated_utils_fedavg_copy_copy import *

 
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [54]:
#declear path to your data
drebin_data_path = r'data\drebin.csv'
malgenome_data_path = r'data\malgenome.csv'
kronodroid_data_path = r'data\kronodroid.csv'
TUANDROMD_data_path=r'data\TUANDROMD.csv'



Drebin_data = pd.read_csv(drebin_data_path, header = None)

Malgenome_data = pd.read_csv(malgenome_data_path)

Tuandromd_data=pd.read_csv(TUANDROMD_data_path)

kronodroid_data=pd.read_csv(kronodroid_data_path)
Kronodroid_data = kronodroid_data.iloc[:,range(1,kronodroid_data.shape[1])]
import random
import numpy as np

def create_clientsss(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list: a list of binarized labels for each image
            num_client: number of federated members (clients)
            initials: the clients' name prefix, e.g., clients_1 
    '''

    # create a list of client names
    client_names = ['{}_{}'.format(initial, i + 1) for i in range(num_clients)]

    # shuffle the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    # shard data and place at each client with non-i.i.d distribution of classes
    shards = [[] for _ in range(num_clients)]
    classes = set(tuple(label) for label in label_list)
    classes_per_client = max(1, len(classes) // num_clients)

    # Count occurrences of each class within each client
    class_counts_per_client = {client_name: {class_tuple: 0 for class_tuple in classes} for client_name in client_names}

    for i in range(num_clients):
        # Convert the set to a list before using random.sample
        class_list = list(classes)
        
        # Select classes for this client
        client_classes = random.sample(class_list, min(classes_per_client, len(class_list)))
        
        # Select data points for the selected classes
        for image, label in data:
            if tuple(label) in client_classes:
                shards[i].append((image, label))
                class_counts_per_client[client_names[i]][tuple(label)] += 1

        # Print the distribution of classes within the client
        class_distribution = class_counts_per_client[client_names[i]]
        print(f"Client {client_names[i]}: {class_distribution}")

    # number of clients must equal the number of shards
    assert len(shards) == len(client_names)

    return {client_names[i]: shards[i] for i in range(len(client_names))}



In [73]:
import numpy as np
import random

def create_clients_non_iid(image_list, label_list, num_clients=10, alpha=5, min_samples_per_client=32, initial='clients'):
    """
    Return a dictionary with keys as clients' names and values as data shards,
    represented as tuples of images and label lists. The data distribution among
    clients is non-identically distributed (non-IID), and each client has at least
    32 data points.

    Args:
        image_list (list): A list of numpy arrays of training images.
        label_list (list): A list of binarized labels for each image.
        num_clients (int): Number of federated members (clients).
        alpha (float): Concentration parameter for the Dirichlet distribution.
        min_samples_per_client (int): Minimum number of data points per client.
        initial (str): The clients' name prefix, e.g., 'clients_1'.

    Returns:
        dict: A dictionary mapping client names to non-IID data shards.
    """

    # Create a list of client names
    client_names = ['{}_{}'.format(initial, i + 1) for i in range(num_clients)]

    # Randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)
    print(len(data))
    # Define a Dirichlet distribution to sample proportions for each client
    proportions = np.random.dirichlet([alpha] * num_clients)
    print("num clients" , num_clients)
    print("proportions " , proportions)
    # Calculate the number of samples each client should get
    total_samples = len(data)
    client_samples = (proportions * total_samples).astype(int)
    print("client_samples " , client_samples)
    # Ensure each client gets at least min_samples_per_client
    for i in range(num_clients):
        if client_samples[i] < min_samples_per_client:
            client_samples[i] = min_samples_per_client

    shards = [data[i:i + size] for i, size in enumerate(client_samples)]

    # Number of clients must equal the number of shards
    assert len(shards) == len(client_names)

    for i in range(num_clients):
        # Print the number of data points for each class in the current client
        class_counts = {cls: sum(1 for _, label in shards[i] if label == cls) for cls in set(label_list)}
        print(f"Client {client_names[i]}: {class_counts}")

    return {client_names[i]: shards[i] for i in range(num_clients)}


In [74]:
import numpy as np
import random

def create_clients_non_iid(image_list, label_list, num_clients=10, min_samples_per_client=32, initial='clients'):
    """
    Return a dictionary with keys as clients' names and values as data shards,
    represented as tuples of images and label lists. The data distribution among
    clients is non-identically distributed (non-IID), and each client has at least
    32 data points.

    Args:
        image_list (list): A list of numpy arrays of training images.
        label_list (list): A list of binarized labels for each image.
        num_clients (int): Number of federated members (clients).
        min_samples_per_client (int): Minimum number of data points per client.
        initial (str): The clients' name prefix, e.g., 'clients_1'.

    Returns:
        dict: A dictionary mapping client names to non-IID data shards.
    """

    # Create a list of client names
    client_names = ['{}_{}'.format(initial, i + 1) for i in range(num_clients)]

    # Randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    shards = []
    for i in range(num_clients):
        # Generate a random alpha value for each client
        alpha = random.uniform(0.1, 5.0)
        
        # Define a Dirichlet distribution to sample proportions for each class within the client
        class_proportions = np.random.dirichlet([alpha] * len(set(label_list)))
        
        # Calculate the number of samples each client should get
        total_samples = len(data)
        client_samples = int((total_samples * 1.0 / num_clients) + 0.5)
        
        # Ensure each client gets at least min_samples_per_client
        client_samples = max(client_samples, min_samples_per_client)
        
        # Sample data for the current client using the class proportions
        client_data = []
        for cls in set(label_list):
            class_samples = int(client_samples * class_proportions[cls] + 0.5)
            class_data = [item for item in data if item[1] == cls][:class_samples]
            client_data.extend(class_data)
        
        shards.append(client_data)

        # Print the number of data points for each class in the current client
        class_counts = {cls: sum(1 for _, label in client_data if label == cls) for cls in set(label_list)}
        print(f"Client {client_names[i]}: {class_counts}")

    # Number of clients must equal the number of shards
    assert len(shards) == len(client_names)

    return {client_names[i]: shards[i] for i in range(num_clients)}

In [79]:
import numpy as np
import random

def create_clients_non_iid3(image_list, label_list, num_clients=10, min_samples_per_client=32, initial='clients'):
    """
    Return a dictionary with keys as clients' names and values as data shards,
    represented as tuples of images and label lists. The data distribution among
    clients is non-identically distributed (non-IID), and each client has at least
    32 data points.

    Args:
        image_list (list): A list of numpy arrays of training images.
        label_list (list): A list of binarized labels for each image.
        num_clients (int): Number of federated members (clients).
        min_samples_per_client (int): Minimum number of data points per client.
        initial (str): The clients' name prefix, e.g., 'clients_1'.

    Returns:
        dict: A dictionary mapping client names to non-IID data shards.
    """

    # Create a list of client names
    client_names = ['{}_{}'.format(initial, i + 1) for i in range(num_clients)]

    # Randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    shards = []
    for i in range(num_clients):
        # Generate a random alpha value for each client
        alpha = random.uniform(0.1, 5.0)
        
        # Define a Dirichlet distribution to sample proportions for each class within the client
        class_proportions = np.random.dirichlet([alpha] * len(set(label_list)))
        
        # Calculate the number of samples each client should get
        total_samples = len(data)
        random_factor = random.uniform(0.5, 1.5)  # Adjust the range as needed
        client_samples = int((total_samples * random_factor / num_clients) + 0.5)
        
        # Ensure each client gets at least min_samples_per_client
        client_samples = max(client_samples, min_samples_per_client)
        
        # Sample data for the current client using the class proportions
        client_data = []
        for cls in set(label_list):
            class_samples = int(client_samples * class_proportions[cls] + 0.5)
            class_data = [item for item in data if item[1] == cls][:class_samples]
            client_data.extend(class_data)
        
        shards.append(client_data)

        # Print the number of data points for each class in the current client
        class_counts = {cls: sum(1 for _, label in client_data if label == cls) for cls in set(label_list)}
        print(f"Client {client_names[i]}: {class_counts}, Total Samples: {len(client_data)}")

    # Number of clients must equal the number of shards
    assert len(shards) == len(client_names)

    return {client_names[i]: shards[i] for i in range(num_clients)}

In [91]:
def create_clients_non_iid3(image_list, label_list, num_clients=10, min_samples_per_client=32, initial='clients'):
    """
    Return a dictionary with keys as clients' names and values as data shards,
    represented as tuples of images and label lists. The data distribution among
    clients is non-identically distributed (non-IID), and each client has at least
    32 data points.

    Args:
        image_list (list): A list of numpy arrays of training images.
        label_list (list): A list of binarized labels for each image.
        num_clients (int): Number of federated members (clients).
        min_samples_per_client (int): Minimum number of data points per client.
        initial (str): The clients' name prefix, e.g., 'clients_1'.

    Returns:
        dict: A dictionary mapping client names to non-IID data shards.
    """

    # Create a list of client names
    client_names = ['{}_{}'.format(initial, i + 1) for i in range(num_clients)]
    # If X_train is a Pandas DataFrame
    num_train_samples = len(image_list)
    print("Number of data points in the training set: ", num_train_samples)

    # Randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    shards = []
    for i in range(num_clients):
        # Generate a random alpha value for each client
        alpha = random.uniform(0.1, 5.0)
        
        # Define a Dirichlet distribution to sample proportions for each class within the client
        class_proportions = np.random.dirichlet([alpha] * len(set(label_list)))
        
        # Calculate the number of samples each client should get
        total_samples = len(data)
        random_factor = random.uniform(0.5, 1.5)  # Adjust the range as needed
        client_samples = int((total_samples * random_factor / num_clients) + 0.5)
        
        # Ensure each client gets at least min_samples_per_client
        client_samples = max(client_samples, min_samples_per_client)
        
        # Sample data for the current client using the class proportions without replacement
        client_data = []
        for cls in set(label_list):
            class_samples = int(client_samples * class_proportions[cls] + 0.5)
            class_data = random.sample([item for item in data if item[1] == cls], min(class_samples, len(data)))
            client_data.extend(class_data)
        
        shards.append(client_data)

        # Print the number of data points for each class in the current client
        class_counts = {cls: sum(1 for _, label in client_data if label == cls) for cls in set(label_list)}
        print(f"Client {client_names[i]}: {class_counts}, Total Samples: {len(client_data)}")

    # Number of clients must equal the number of shards
    assert len(shards) == len(client_names)

    return {client_names[i]: shards[i] for i in range(num_clients)}


In [110]:
import numpy as np
import random
from collections import defaultdict

def create_clients_non_iid3(image_list, label_list, num_clients=10, min_samples_per_client=32, initial='clients'):
    """
    Return a dictionary with keys as clients' names and values as data shards,
    represented as tuples of images and label lists. The data distribution among
    clients is non-identically distributed (non-IID), and each client has at least
    32 data points.

    Args:
        image_list (list): A list of numpy arrays of training images.
        label_list (list): A list of binarized labels for each image.
        num_clients (int): Number of federated members (clients).
        min_samples_per_client (int): Minimum number of data points per client.
        initial (str): The clients' name prefix, e.g., 'clients_1'.

    Returns:
        dict: A dictionary mapping client names to non-IID data shards.
    """

    # Create a list of client names
    client_names = ['{}_{}'.format(initial, i + 1) for i in range(num_clients)]

    # Randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    shards = []
    for i in range(num_clients - 1):  # For all clients except the last one
        # Generate a random alpha value for each client
        alpha = random.uniform(0.1, 5.0)
        
        # Define a Dirichlet distribution to sample proportions for each class within the client
        class_proportions = np.random.dirichlet([alpha] * len(set(label_list)))
        
        # Calculate the number of samples each client should get
        total_samples = len(data)
        random_factor = random.uniform(0.5, 1.5)  # Adjust the range as needed
        client_samples = int(min_samples_per_client + (total_samples - num_clients * min_samples_per_client) * random_factor)
        
        # Ensure each client gets at least min_samples_per_client
        client_samples = max(client_samples, min_samples_per_client)
        
        # Sample data for the current client using the class proportions without replacement
        client_data = []
        for cls in set(label_list):
            class_samples = int(client_samples * class_proportions[cls] + 0.5)
            class_data = random.sample([item for item in data if item[1] == cls], max(0, min(class_samples, len(data))))
            client_data.extend(class_data)

        # Print the number of data points for each class in the current client
        class_counts = {cls: sum(1 for _, label in client_data if label == cls) for cls in set(label_list)}
        print(f"Client {client_names[i]}: {class_counts}, Total Samples: {len(client_data)}")

        # Remove sampled data from the main list
        data = [item for item in data if not any(np.array_equal(item[0], client_item[0]) and item[1] == client_item[1] for client_item in client_data)]

    # The last client gets all the remaining data
    shards.append(data)

    # Print the number of data points for each class in the last client
    class_counts = {cls: sum(1 for _, label in data if label == cls) for cls in set(label_list)}
    print(f"Client {client_names[-1]}: {class_counts}, Total Samples: {len(data)}")

    # Number of clients must equal the number of shards
    assert len(shards) == len(client_names)

    return {client_names[i]: shards[i] for i in range(num_clients)}


## fedavg

In [111]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

def train_model(model, train_loader, loss_fn, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

all_avg = []
all_std = []

n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']
for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # If X_train is a Pandas DataFrame
            num_train_samples = len(X_train)

            print("Number of data points in the training set: ", num_train_samples)




            # create clients -- Horizontal FL
            clients = create_clients_non_iid3(X_train, [tuple(label) for label in y_train.astype(int).tolist()], num_clients=number_of_clients, initial='client')


            # Calculate total number of data points
            total_data_points = sum(len(data) for data in clients.values())
            print(f"Total number of data points: {total_data_points}")
            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedAvg 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------

            print('|=======================|')
            print('|Traditional FedAvg 2017|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]

                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)


                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    train_model(local_model, train_loader, loss, optimizer)


                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()
                
                # ...
                
                # to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedAvg-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedAvg-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedAvg-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 10
No. of Rounds: 100
---------------------------------------------
Number of data points in the training set:  12028


ValueError: Sample larger than population or is negative